In [2]:
import pickle
import pandas as pd
from collections import defaultdict
import ast

# For Josh: Remove duplicates

In [3]:
f = open("allads_vaccine", "rb")
ads = pickle.load(f)
f.close()

In [4]:
f = open("allads_election", "rb")
ads_election = pickle.load(f)
f.close()

In [5]:
df = pd.DataFrame.from_records(ads)
df['ad_creative_bodies'] = df['ad_creative_bodies'].str[0]
df=df.drop_duplicates(subset=['ad_creative_bodies'])
df=df.sample(frac=1)

In [8]:
df_election = pd.DataFrame.from_records(ads_election)
df_election['ad_creative_bodies'] = df_election['ad_creative_bodies'].str[0]
df_election=df_election.drop_duplicates(subset=['ad_creative_bodies'])
df_election=df_election.sample(frac=1)

In [9]:
df.to_csv('Ads_vaccine_sample.csv')

In [10]:
df_election.to_csv('Ads_election_sample.csv')

# For emily: combine duplicates

In [14]:
f = open("allads_election", "rb")
ads_emily_election = pickle.load(f)
f.close()

In [29]:
f = open("allads_vaccine", "rb")
ads_emily_vaccine = pickle.load(f)
f.close()

28024

In [13]:
def clean(ads_emily):
    df2 = pd.DataFrame.from_records(ads_emily)
    df2=df2.drop(['id', 'ad_delivery_start_time', 'ad_delivery_stop_time', 'ad_creation_time'], axis=1)
    df2 = df2.dropna(subset=['delivery_by_region'])
    df2['ad_creative_bodies'] = df2['ad_creative_bodies'].str[0]
    df2_grouped = df2.groupby('ad_creative_bodies').agg({'demographic_distribution':lambda x: list(x), 
                                           'delivery_by_region':lambda x: list(x), 
                                          'demographic_distribution':lambda x: list(x), 
                                          'estimated_audience_size':lambda x: list(x), 
                                          'spend': lambda x: list(x), 
                                          'impressions': lambda x: list(x)}).reset_index()
    
    return df2_grouped
    

In [22]:
df2_grouped = clean(ads_emily_vaccine)
df2_grouped.head()

,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions
0,,"[[{'percentage': '0.001006', 'age': '13-17', '...","[[{'percentage': '0.001516', 'region': 'Rhode ...","[{'lower_bound': '5001', 'upper_bound': '10000...","[{'lower_bound': '1000', 'upper_bound': '1499'...","[{'lower_bound': '25000', 'upper_bound': '2999..."
1,!!!!!!!!!!!!!!!!!!!!,"[[{'percentage': '4.9E-5', 'age': '18-24', 'ge...","[[{'percentage': '0.034184', 'region': 'Alabam...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '25000', 'upper_bound': '2999..."
2,"""'Safe, potent, & effective' - those three wor...","[[{'percentage': '5.4E-5', 'age': '13-17', 'ge...","[[{'percentage': '0.000163', 'region': 'Unknow...",[{'lower_bound': '1000001'}],"[{'lower_bound': '300', 'upper_bound': '399'}]","[{'lower_bound': '15000', 'upper_bound': '1999..."
3,"""1 in 500 Americans have died from COVID-19. O...","[[{'percentage': '0.000943', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Texas'}], [{'...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '1000', 'upper_bound': '1999'..."
4,"""A patent seizure is largely symbolic rather t...","[[{'percentage': '0.000456', 'age': '18-24', '...","[[{'percentage': '0.01887', 'region': 'Alabama...",[{'lower_bound': '1000001'}],"[{'lower_bound': '200', 'upper_bound': '299'}]","[{'lower_bound': '20000', 'upper_bound': '2499..."


In [23]:
def get_cumulative_spend(df2_grouped):
    cum_impressions = []
    cum_spend = []
    for row in df2_grouped.iterrows():
        ind_region = row[1]['delivery_by_region']
        ind_impressions = row[1]['impressions']
        ind_spend = row[1]['spend']
        cur_cum_impressions = defaultdict(int)
        cur_cum_spend = 0
        for i in range(len(ind_region)): 
            cur_spend =(float(ind_spend[i]['lower_bound']) + float(ind_spend[i]['upper_bound']))/2
            cur_cum_spend = cur_cum_spend+cur_spend
            region = ind_region[i]
            impr = ind_impressions[i]
            for key in region:    
                old = cur_cum_impressions[key['region']]
                region_name = key['region']
                try: 
                    cur_cum_impressions[region_name] = old+(float(impr['lower_bound']) + float(impr['upper_bound']))/2
                except:
                    cur_cum_impressions[region_name] = old+(float(impr['lower_bound']))/2

        cum_impressions.append(dict(cur_cum_impressions))
        
        cum_spend.append(cur_cum_spend)
    return cum_spend, cum_impressions

In [26]:
cum_spend, cum_impressions = get_cumulative_spend(df2_grouped)
df2_grouped['cumulative_impressions_by_region'] = cum_impressions
df2_grouped['cum_spend'] = cum_spend
df2_grouped.head()

,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions,cumulative_impressions_by_region,cum_spend
0,,"[[{'percentage': '0.001006', 'age': '13-17', '...","[[{'percentage': '0.001516', 'region': 'Rhode ...","[{'lower_bound': '5001', 'upper_bound': '10000...","[{'lower_bound': '1000', 'upper_bound': '1499'...","[{'lower_bound': '25000', 'upper_bound': '2999...","{'Rhode Island': 2231992.5, 'Oregon': 2231992....",48641.0
1,!!!!!!!!!!!!!!!!!!!!,"[[{'percentage': '4.9E-5', 'age': '18-24', 'ge...","[[{'percentage': '0.034184', 'region': 'Alabam...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '25000', 'upper_bound': '2999...","{'Alabama': 27499.5, 'Rhode Island': 27499.5, ...",49.5
2,"""'Safe, potent, & effective' - those three wor...","[[{'percentage': '5.4E-5', 'age': '13-17', 'ge...","[[{'percentage': '0.000163', 'region': 'Unknow...",[{'lower_bound': '1000001'}],"[{'lower_bound': '300', 'upper_bound': '399'}]","[{'lower_bound': '15000', 'upper_bound': '1999...","{'Unknown': 17499.5, 'Pennsylvania': 17499.5, ...",349.5
3,"""1 in 500 Americans have died from COVID-19. O...","[[{'percentage': '0.000943', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Texas'}], [{'...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '1000', 'upper_bound': '1999'...",{'Texas': 17496.5},346.5
4,"""A patent seizure is largely symbolic rather t...","[[{'percentage': '0.000456', 'age': '18-24', '...","[[{'percentage': '0.01887', 'region': 'Alabama...",[{'lower_bound': '1000001'}],"[{'lower_bound': '200', 'upper_bound': '299'}]","[{'lower_bound': '20000', 'upper_bound': '2499...","{'Alabama': 22499.5, 'Rhode Island': 22499.5, ...",249.5


In [27]:
df2_grouped.to_csv('emily_vaccine.csv')

In [28]:
df2_grouped.shape

(7248, 8)